In [1]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "iaraya"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

In [2]:
import random

def write_instance(L,W,H,boxes,filename) :
    txt = "1\n1 0\n"+str(L)+" "+str(W)+" "+str(H)+"\n";
    txt += str(len(boxes)) + "\n"
    for box in boxes:
        txt += str(box.id) + ' ' + str(box.l) + ' ' + str(box.rotx) \
            + ' ' + str(box.w) + ' ' + str(box.roty) \
            + ' ' + str(box.h) + ' ' + str(box.rotz) + ' ' + str(boxes[box])  + '\n'
    txt += '\n'

    text_file = open(filename, "w")
    text_file.write(txt)
    text_file.close()


In [3]:
import os
import json
from scp import SCPClient

def bsg_solve(L,W,H,boxes,id2box, time=1):
    filename="tmp_instance_"+ str(random.randint(10000, 99999))
    
    write_instance(L,W,H,boxes,filename)
    
    scp = SCPClient(ssh.get_transport())
    scp.put(filename, "/home/iaraya/clp/"+filename)

    stdin, stdout, stderr = ssh.exec_command("/home/iaraya/clp/BSG_CLP /home/iaraya/clp/"+filename+" -i 0 -t "+str(time)+" --json")
    lines = stdout.readlines()
    print(lines[-1])
    json_data = json.loads(lines[-1])
    loaded={}; remaining={}
    for item in json_data["loaded"]:
        loaded[id2box[item[0]]]=item[1]
    for item in json_data["remaining"]:
        remaining[id2box[item[0]]]=item[1]
        
    ssh.exec_command("rm /home/iaraya/clp/"+filename)
    os.remove(filename)
    return remaining, loaded, json_data["utilization"]


In [4]:
from box import box
import random
# id l   w g  rotx roty rotz nn
# 1	 108 0 76 0	   30	1	 7

def load_BRinstance(filename, inst=1, nbox = 1):
    boxes={}; id2box={}
    with open(filename) as f:
        n = int(next(f))
        for i in range(n):
            next(f)
            L, W, H =  [int(x) for x in next(f).split()]
            n_boxes = int(next(f))
            for it in range(n_boxes):
                id, l, rotx, w, roty, h, rotz, nn = [int(x) for x in next(f).split()] 
                if i == inst-1:
                    b = box(id, l, w, h, rotx, roty, rotz)
                    # Multiplicar nn para complejizar el problema
                    boxes[b]=nn*nbox;
                    id2box[id]=b
            if i == inst-1: 
                return L,W,H,boxes,id2box

### Ejemplo usando instancia BR

In [5]:
from bin import bin
import numpy as np

L,W,H,boxes,id2box = load_BRinstance(filename="BR8.txt",inst=4, nbox=10)

#Dimension del contenedor
L = int(L/2)
W = int(W/2)
H = int(H/2)

Vmax = L*W*H

In [6]:
def generate_candidate_solution(r_param = 1.0) -> list:
    solution_size = 0
    solution = np.array([])

    #Generating candidate Solution
    while(len(boxes) > 0):
        solution_size +=1 
        vol_c = 0
        c = dict()

        #Llenado del contenedor
        while(vol_c < r_param*Vmax and len(boxes) != 0):
            boxes_keys = list(boxes.keys())
            r =  random.randint(0, len(boxes_keys)-1)
            b = boxes_keys[r]
            n = min(boxes[b],8)    
            boxes[b] = boxes[b]-n

            # No quedan mas cajas del tipo b
            if(boxes[b] == 0):
                boxes.pop(b)

            #Agregar cantidad de cajas
            if(b not in c):
                c[b] = n
            else:
                c[b] = c[b]+n
            
            vol_box = b.vol
            vol_c = vol_c + vol_box*n

        #Evaluar contenedor cajas obtenidas 
        remaining, loaded, utilization = bsg_solve(L,W,H, c , id2box, time=5)
        container = bin(solution_size, loaded, utilization)
        #Se agrega la solucion
        solution = np.append(solution, container)
        
        # añadiendo las restantes para ser distribuidas nuevamente
        for key in remaining:
            if(key in boxes):
                boxes[key] += remaining[key]
            else:
                boxes[key] = remaining[key]

    print("Initial Solution: {}".format(len(solution)))
    return solution

In [7]:
def print_solution(solution:list):
    sol_size = 0
    for s in solution:
        
        vol = get_vol_by_boxes_group(boxes = s.boxes)
        if vol != 0.0 :
            print("id: {} - vol: {}".format(s.id, vol))
            sol_size+=1
        
    print("Solution Size: {}".format(sol_size))

### Ejemplo creando instancia y resolviendo

In [8]:
# Retorna la probabilidad de llenar el
def calculate_prob(solution: list, metric:float) -> float:
    counter = 0
    size = 0
    for s in solution:
        if(len(s.boxes) != 0):
            size+=1
            v = get_vol_by_boxes_group(s.boxes)
            if(v >= metric):
                counter += 1


    prob_calculate = (float)(counter/size)
    return prob_calculate

In [9]:
def get_media_volumen(solution:list) -> float:
    ponderate = 0
    size = 0

    for s in solution:
        bin_boxes = s.boxes
        if(len(bin_boxes) != 0):
            ponderate += get_vol_by_boxes_group(bin_boxes)
            size+=1 
    
    media = float(ponderate/size)
    return media    

In [10]:
def get_vol_by_boxes_group(boxes: dict) -> float:
    vol = 0.0
    for box in boxes:
        vol += box.vol*boxes[box]
    final_vol = float(vol/Vmax)
    
    return final_vol

In [11]:
def get_random_bin(s:list, b:bin = None) -> object:
    
    get_id = int(random.randint(0, len(s)-1))
    bin_selected = s[get_id]

    #Se retorna un bin distinto al seleccionado y que no este vacio
    while(b == bin_selected or len(bin_selected.boxes) == 0):
        get_id = int(random.randint(0, len(s)-1))
        bin_selected = s[get_id]
    
    bin_selected.verify = False
    return bin_selected


In [12]:
def pop_random_boxes_from_bin(last_bin: object, cant_boxes: int) -> dict:
    boxes_to_share = dict()
    boxes = last_bin.boxes

    for i in range(cant_boxes):
        total_keys = list(boxes.keys())
        
        #Selecciona una caja aleatoria
        get_id = int(random.randint(0, len(total_keys)-1))
        box = total_keys[get_id]
        boxes[box] = boxes[box]-1

        # Agrega la caja al conjunto
        if(box not in boxes_to_share):
            boxes_to_share[box] = 1
        else:
            boxes_to_share[box] = boxes_to_share[box]+1
        
        # Elimina la caja del conjunto si no quedan mas
        if(boxes[box] == 0):
            boxes.pop(box)
        
        if(len(boxes) == 0):
            break
    
    return boxes_to_share
    

In [13]:
def eval_list_bins(solution:list, media) -> float:
    quality = 0.0

    for b in solution:
        bin_boxes = b.boxes
        diff = get_vol_by_boxes_group(bin_boxes) - media
        quality += pow(diff,2)

    return quality


In [14]:
def predict_vol(boxes:dict, box: object) -> float:
    vol_box = box.vol
    vol_bin = get_vol_by_boxes_group(boxes)
    total_vol = float(vol_box/Vmax) + vol_bin
    
    return total_vol


In [15]:
def verify_solution(solution: list) -> bool:
    factibility = True
    for s in solution:
        if(not(s.verify)):
            s.verify = True
            boxes = s.boxes
            persistens = True
            while(persistens):
                try:
                    remaining, loaded, utilization = bsg_solve(L,W,H, boxes, id2box, time=10)
                    persistens = False
                except:
                    persistens = True

            if(len(remaining) != 0):
                return False
                
    return factibility


In [16]:
def random_swap(solution:list, media_volumen:float):
    # Seleccionar bin 1 y 2 
    bin_1 = get_random_bin(s = solution) 
    bin_2 = get_random_bin(s = solution, b = bin_1)
    
    v1 = get_vol_by_boxes_group(bin_1.boxes)
    v2 = get_vol_by_boxes_group(bin_2.boxes)

    var_initial = eval_list_bins([bin_1, bin_2], media_volumen)

    print("Initial ")
    print("Bin 1: {} ".format(v1))
    print("Bin 2: {} ".format(v2))

    # Seleccionar boxes del bin 1
    total_boxes = int(random.randint(1, 2))
    boxes_1 = pop_random_boxes_from_bin(last_bin=bin_1, cant_boxes=total_boxes)

    # Se agregan cajas de bin_1 en bin_2
    for box in boxes_1:
        if(box not in bin_2.boxes):
            bin_2.boxes[box] = boxes_1[box]
        else:
            bin_2.boxes[box] = bin_2.boxes[box] + boxes_1[box]
    
    v2 = get_vol_by_boxes_group(bin_2.boxes)
    
    # Probabilidad de poder llenar el bin 2
    # Probabilidad = Proporcion de bins con volumen mayor a v2

    p2 = calculate_prob(solution, v2)
    # Seleccionar cajas desde el bin b_2
    while( p2 <= random.random() ):
            
        v1 = get_vol_by_boxes_group(bin_1.boxes)
        v2 = get_vol_by_boxes_group(bin_2.boxes)
        p1 = calculate_prob(solution, v1)
        
        # Se espera solo una caja desde bin_2
        boxes_2 = pop_random_boxes_from_bin(last_bin=bin_2, cant_boxes= 1)
        box_2 = list(boxes_2.keys())[0] 

        if(predict_vol(bin_1.boxes, box_2) > 1 ):
            # print("La caja escogida sobrepasa el máximo del bin 1")
            return None, -1  

        # Se agrega la caja al conjunto del bin2 
        if(box_2 not in bin_1.boxes):  
            bin_1.boxes[box_2] = 1
        else:
            bin_1.boxes[box_2] = bin_1.boxes[box_2] + 1

        v1 = get_vol_by_boxes_group(bin_1.boxes)
        v2 = get_vol_by_boxes_group(bin_2.boxes)
        p2 = calculate_prob(solution, v2)
        p1 = calculate_prob(solution, v1) # -> 0: No se puede llenar

        if( p1 <= random.random()):
            return None, -1

        if(len(bin_2.boxes) == 0):
            # print("Empty bin 2")
            break
    
    if(v2 > 1):
        # print("Sobrepasa Volumen del bin 2")
        return None, -1
    else:
        print("After Swap")
        v1 = get_vol_by_boxes_group(bin_1.boxes)
        v2 = get_vol_by_boxes_group(bin_2.boxes)                  
        
        var_final = eval_list_bins([bin_1, bin_2], media_volumen)
        eval = var_final - var_initial
        print("Bin 1: {} ".format(v1))
        print("Bin 2: {} ".format(v2))
        print("Evaluation: {}. Varianza Inicial: {}. Varianza final: {}".format(eval, var_initial, var_final ))

        return solution, eval

In [17]:
import copy as cp

def mclp(initial_solution: list, cant_test: int) -> list: 
    results = {"factible": 0, "unfactible": 0, "swapped":0, "worst":0}
    best_solution = cp.deepcopy(initial_solution)
    media = get_media_volumen(initial_solution)

    for i in range(cant_test):
        current_solution = cp.deepcopy(best_solution)
        current_solution, current_eval = random_swap(solution = current_solution, media_volumen = media)
        
        if(current_solution is not None):
            print("--------------------- Swapping {} -------------------- ".format(i))
            results["swapped"]+=1
            if(current_eval > 0):   
                if(verify_solution(current_solution)):
                    results["factible"]+=1
                    print("Nueva factible")
                    best_solution = cp.deepcopy(current_solution)
                else:
                    results["unfactible"]+=1
                    print("Solucion infactible")
            else:
                results["worst"]+=1
                print("La solucion no mejora")
        
    print(results)
    return best_solution


r_param = 1.5 
total_test = 100

#Creacion de solucion candidata
init_solution = generate_candidate_solution(r_param=r_param)
swapped_solution = mclp(initial_solution=init_solution, cant_test = total_test)


{"remaining" :[[30,1],[21,6],[13,4]], "loaded" :[[30,7],[15,8],[21,2],[13,4]], "utilization" : 0.86363503}

{"remaining" :[[9,3],[6,3],[17,8]], "loaded" :[[2,8],[9,5],[6,5]], "utilization" : 0.79758391}

{"remaining" :[[2,5],[4,2],[26,4],[16,1],[17,5]], "loaded" :[[2,11],[4,6],[26,4],[16,7],[17,3]], "utilization" : 0.92267752}

{"remaining" :[[19,7],[14,5]], "loaded" :[[23,8],[19,1],[14,3]], "utilization" : 0.88007184}

{"remaining" :[[5,8],[18,7],[9,5],[11,3],[4,2],[13,3]], "loaded" :[[18,1],[9,3],[11,5],[4,6],[13,5]], "utilization" : 0.92687259}

{"remaining" :[[3,2],[8,7],[7,8],[27,8]], "loaded" :[[3,6],[9,8],[8,9]], "utilization" : 0.87767956}

{"remaining" :[[16,8],[29,4],[26,8]], "loaded" :[[29,4],[13,8]], "utilization" : 0.89994544}

{"remaining" :[[8,2],[6,8],[5,2]], "loaded" :[[8,6],[2,8],[7,8],[5,6]], "utilization" : 0.91109322}

{"remaining" :[[17,6],[4,1],[21,3],[14,7]], "loaded" :[[17,2],[4,7],[21,5],[14,1]], "utilization" : 0.82987418}

{"remaining" :[[22,2],[8,16],[9,3],

In [18]:
print("Initial Solution")
print_solution(init_solution)
print("Swapped Solution")
print_solution(swapped_solution)

Initial Solution
id: 1 - vol: 0.8636350262659548
id: 2 - vol: 0.7975839066194486
id: 3 - vol: 0.9226775225480651
id: 4 - vol: 0.8800718435383611
id: 5 - vol: 0.9268725860464121
id: 6 - vol: 0.8776795553510864
id: 7 - vol: 0.8999454352873206
id: 8 - vol: 0.9110932200669755
id: 9 - vol: 0.8298741801919394
id: 10 - vol: 0.9275795200445077
id: 11 - vol: 0.8711127991697605
id: 12 - vol: 0.9417243519102999
id: 13 - vol: 0.7120288444049772
id: 14 - vol: 0.8509110167224796
id: 15 - vol: 0.9411139760557202
id: 16 - vol: 0.8550370719077321
id: 17 - vol: 0.800302780660554
id: 18 - vol: 0.8620662907764237
id: 19 - vol: 0.9073905763531513
id: 20 - vol: 0.9630559448789412
id: 21 - vol: 0.8455091101672247
id: 22 - vol: 0.9418099436164635
id: 23 - vol: 0.8944408186846694
id: 24 - vol: 0.9107257106786353
id: 25 - vol: 0.8294692244321525
id: 26 - vol: 0.8671266329292692
id: 27 - vol: 0.8874939818331604
id: 28 - vol: 0.9461475708003895
id: 29 - vol: 0.8760107845549766
id: 30 - vol: 0.8984106690061733
id:

In [27]:
print("Volumen Contenedor: ", Vmax)
for b in swapped_solution:
    print(b.boxes, b.utilization)
    for box in b.boxes.keys():
        print(box.vol)

Volumen Contenedor:  3738680
{<box.box object at 0x7f5808936610>: 7, <box.box object at 0x7f5808936590>: 8, <box.box object at 0x7f5808936250>: 2, <box.box object at 0x7f5808936b50>: 4} 0.86363503
147609
41943
289800
320112
{<box.box object at 0x7f58089369d0>: 8, <box.box object at 0x7f5808936550>: 5, <box.box object at 0x7f5808936b90>: 5} 0.79758391
53872
236368
273819
{<box.box object at 0x7f58089369d0>: 11, <box.box object at 0x7f5808936050>: 6, <box.box object at 0x7f5808936bd0>: 4, <box.box object at 0x7f5808936310>: 7, <box.box object at 0x7f5808936b10>: 3} 0.92267752
53872
109890
246895
70210
239538
{<box.box object at 0x7f5808936990>: 8, <box.box object at 0x7f5808936c90>: 1, <box.box object at 0x7f5808936d10>: 3} 0.88007184
215760
348243
405328
{<box.box object at 0x7f5808936e10>: 1, <box.box object at 0x7f5808936550>: 3, <box.box object at 0x7f5808936e90>: 5, <box.box object at 0x7f5808936050>: 6, <box.box object at 0x7f5808936b50>: 5} 0.92687259
76296
236368
83996
109890
320